## お手軽 Bag of Visual Words (BoVW)

### 概要
Bag of Features (BoF) とも呼ばれます。
**簡易的な実装**なので、精度・速度ともに実用には不向きです。多くの部分で並列処理などを実装していないので遅いです。気長に実行してください。（Colabのタイムアウトに注意してください）。

BoVW参考：

* https://hazm.at/mox/machine-learning/computer-vision/recipes/similar-image-retrieval.html
* https://n-hidekey.hatenadiary.org/entry/20111120/1321803326

### 準備
**GBDTの学習にGPUを使う場合はGPUインスタンスで実行してください：**
1.   「ランタイム」から「ランタイムのタイプを変更」
2.   ハードウェアアクセラレータを「GPU」に

途中でキャッシュされるpickleファイル（'*.pkl'）をダウンロードしておき、次回のランタイム起動後にアップロードして使うと、時間のかかる処理をスキップできます。

pklファイルおよびサンプル画像などは、githubリポジトリ`ou_dip/bovw/`以下にありますので、ランタイム起動後にアップロードしてください。




In [ ]:
## 大量データにアクセスする場合、Googleドライブ上だと遅くなるのでコメントアウトしています
# Googleドライブへのマウント（Colab用コード）
# from google.colab import drive
# drive.mount('/content/drive')
# %cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

# 伝統的な方法に則って、SIFT特徴量を使ってみる（2019年に特許が切れたため、最新版にはSIFTが入っている）
!pip install opencv-contrib-python==4.5.4.60

# GBDTのライブラリ（CatBoost）のインストール
!pip install catboost

import cv2
import numpy as np  
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import time
import pickle
import os

def imshow(img):
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

## Step 0: 入力データの準備
ここでは、Caltech101というデータセットを使う。101クラスのデータセットで、およそ10000枚の画像が含まれる。

http://www.vision.caltech.edu/Image_Datasets/Caltech101/

ここでは、8割をtrain、1割をvalidation、1割をtestにランダムに分割する。ここで、各データセットは以下のものを指す（資料によっては呼び方が違ったりする）

* training datasetは、機械学習器の学習に使う
* validation datasetは、学習器のハイパーパラメータ（繰り返し回数など）選択などに使う。
* test datasetは、学習器の精度評価などに使う。**注意：ハイパーパラメータ探索にtestデータセットを使ってはいけない！**

データのダウンロードには、PyTorch（とtorchvision）という深層学習向けライブラリを使っている。深層学習用途じゃなくても、代表的なデータセットの自動ダウンロードなどができるので楽。

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Caltech101データセットのダウンロード
dataset = torchvision.datasets.Caltech101(root='./data', download=True, transform=None)
categories = dataset.categories # category list

# train, val, testデータセットの作成（ランダム分割）
if os.path.exists('datasets.pkl'):  # 各データセットに含まれるファイルのIDが保存されている
  print("Loading datasets from 'datasts.pkl'.")
  with open('datasets.pkl','rb') as f:  
    train_dataset, val_dataset, test_dataset = pickle.load(f)

else:
  n_samples = len(dataset)
  train_size = int(n_samples * 0.8)
  val_size = int((n_samples-train_size)*0.5)
  test_size = n_samples - train_size - val_size
  print("train_size:", train_size, "val_size:", val_size, "test_size", test_size)

  # split dataset into training and test datasets with fixed random seed (42)
  trainval_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size+val_size, test_size], generator=torch.Generator().manual_seed(42))
  train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

  with open('datasets.pkl', 'wb') as f:
      pickle.dump([train_dataset, val_dataset, test_dataset], f)

In [ ]:
# 特定の画像の取り出し関数 (numpy array, grayscale)。データセットと画像IDを渡す
def single_image_loader(dataset,img_id):
  img = np.array(dataset[img_id][0])
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  label_id = dataset[img_id][1]

  return img, label_id

# グリッド点上にキーポイントを作る関数
def create_dense_keypoints(image, detector, step, scale):
    if len(image.shape) == 3:
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
      gray_image = image

    # Create dense keypoints
    keypoints = []
    rows, cols = gray_image.shape
    for y in range(0, rows, step):
      for x in range(0, cols, step):
        keypoints.append(cv2.KeyPoint(float(x), float(y), scale))
    
    return keypoints

# ランダムな点上にキーポイントを作る関数
import random
def create_random_keypoints(image, detector, n_points, scale):
    if len(image.shape) == 3:
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
      gray_image = image

    # Create random keypoints
    keypoints = []
    rows, cols = gray_image.shape
    for i in range(n_points):  
      y = random.randrange(rows)
      x = random.randrange(cols)
      keypoints.append(cv2.KeyPoint(float(x), float(y), scale))
    
    return keypoints

### 共通の設定

In [ ]:
from tqdm.notebook import tqdm
import random

# visual wordsのクラスタ数
n_clusters = 1000

# feat_step画素ごとのグリッド点上で特徴量を計算
feat_step = 9  

# 使用する特徴量
# feat = cv2.ORB_create()  # ORB特徴量
feat = cv2.SIFT_create()  # SIFT特徴量 (OpenCV4.4.0以降)

In [ ]:
# 参考：特徴点の可視化
img_id = 0
img, label_id = single_image_loader(train_dataset,img_id)
print(categories[label_id], ", category id:",label_id)
imshow(img)

print("Dense keypoints")
kp = create_dense_keypoints(img,feat,step=feat_step,scale=feat_step)
img2 = cv2.drawKeypoints(img,kp,None,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
imshow(img2)

print("Random keypoints")
kp = create_random_keypoints(img,feat,n_points=100,scale=feat_step)
kp, desc = feat.compute(img, kp)
img2 = cv2.drawKeypoints(img,kp,None,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
imshow(img2)

## Step 1: Visual Wordsの作成

SIFT特徴量（などの特徴量）のクラスタリングにより、Visual Words（辞書）を作成する。一つのクラスタが一つの「単語」となり、画像（文書）中での「単語」の出現頻度を使って分類などを行う。


In [ ]:
if os.path.exists('vw.pkl'):
  with open('vw.pkl','rb') as f:  
    clusters = pickle.load(f)
  print("Loading", clusters.shape[0], "visual words from 'vw.pkl'.")

else:
  # 時間がかかるので、ランダムにサンプルする。
  # 学習用画像が6000枚くらいあるので、各20点くらいサンプリングすることにする。
  bowTrainer = cv2.BOWKMeansTrainer(n_clusters)

  print("Extracting features from",len(train_dataset),"images...")
  start = time.perf_counter()
  for img_id in tqdm(range(len(train_dataset))):  
    img, label_id = single_image_loader(train_dataset,img_id)

    keypoints = create_random_keypoints(img,feat,n_points=20,scale=feat_step)
    keypoints, descriptors = feat.compute(img, keypoints)

    bowTrainer.add(descriptors.astype(np.float32))

  print("Elapsed time:",time.perf_counter()-start,"[s]")

  # 時間がかかるのでしばし待つ。
  print("Clustering",bowTrainer.descriptorsCount(),"features.")
  start = time.perf_counter()
  clusters = bowTrainer.cluster() # k-means法によるクラスタリング
  print("Elapsed time:",time.perf_counter()-start,"[s]")

  with open('vw.pkl', 'wb') as f:
    pickle.dump(clusters, f)

In [ ]:
print(clusters.shape) # クラスタ数n_clustersの重心：SIFT特徴の場合128次元の特徴空間上に定義される
print(clusters)

## Step 2: 学習データのBoVWヒストグラム特徴の作成

学習データ（とvalidationデータ）の各画像についてVisual wordsのヒストグラム（各画像で、どのクラスタに当てはまる点が何点あるかのヒストグラム）を作成する。

1. 画像中のグリッド上の点で特徴量（SIFTなど）を計算
2. Visual wordsのどの単語に当てはまるか計算（最近傍法）
3. Visual words出現確率のヒストグラムを生成→これをBoVW特徴とする




In [ ]:
# BoVWの特徴量計算クラス（OpenCV）の設定
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = {}
matcher = cv2.FlannBasedMatcher(index_params, search_params)

# bag of visual words extractor
bowExtractor = cv2.BOWImgDescriptorExtractor(feat, matcher)
bowExtractor.setVocabulary(clusters)

# データセット全画像のBoVWヒストグラム（と対応する出力）を計算する関数
def calc_probs(dataset):
  probs = []
  labels = []
  for img_id in tqdm(range(len(dataset))):  
    img, label_id = single_image_loader(dataset,img_id)
    
    # グリッド点上に特徴点を定義
    keypoints = create_dense_keypoints(img,feat,step=feat_step,scale=feat_step)

    # SIFT特徴量を計算し、一番近いクラスタを見つけ（FLANNによる近似最近傍探索）、画像中の全点でヒストグラムを作る
    histogram = bowExtractor.compute(img, keypoints)[0]

    probs.append(histogram) 
    labels.append(label_id) 
  return probs, labels

# train, val各データセットに対してBoVW特徴を計算
if os.path.exists('bovw_train_val.pkl'):
  print("Loading  from 'bovw_train_val.pkl'.")
  with open('bovw_train_val.pkl','rb') as f:  
    probs_train, labels_train, probs_val, labels_val = pickle.load(f)

else:
  print("Extracting BoVW histograms from training dataset")
  start = time.perf_counter()
  probs_train, labels_train = calc_probs(train_dataset) # TRAINING dataset
  print("Elapsed time:",time.perf_counter()-start,"[s]")

  print("Extracting BoVW histograms from validation dataset")
  start = time.perf_counter()
  probs_val, labels_val = calc_probs(val_dataset) # VALIDATION dataset
  print("Elapsed time:",time.perf_counter()-start,"[s]")
  
  with open('bovw_train_val.pkl', 'wb') as f:
    pickle.dump([probs_train, labels_train, probs_val, labels_val], f)

In [ ]:
# ヒストグラム（0番目の画像）の可視化
left = np.arange(0,1000)
plt.bar(left,probs_train[0])

## Step 3: GBDTの学習

メジャーなGBDTのライブラリとして[XGBoost](https://xgboost.readthedocs.io/en/stable/), [LightGBM](https://lightgbm.readthedocs.io/en/latest/), [CatBoost](https://catboost.ai/en/docs/)などがあります。

ここでは、（Colab上でGPUを使うのが最も簡単、という理由で）CatBoostを使うことにします。

損失関数にはデフォルト設定のlog lossを使っています。いわゆる cross entropy lossと同じものです。

参考：

* https://catboost.ai/en/docs/concepts/python-usages-examples
* https://colab.research.google.com/github/catboost/tutorials/blob/master/tools/google_colaboratory_cpu_vs_gpu_tutorial.ipynb

In [ ]:
from catboost import CatBoostClassifier

x_train = np.vstack(probs_train)
x_val = np.vstack(probs_val)

y_train = labels_train
y_val = labels_val
 
print("Training GBDT")
start = time.perf_counter()
    
gbm = CatBoostClassifier(
  iterations=1000,    # イテレーション数
  learning_rate=0.1,  # 学習率
  task_type='GPU'     # CPUインスタンスを使う場合はここをコメントアウト
)

gbm.fit(
    x_train, y_train,           # 学習データ（入力と出力の組）
    eval_set=(x_val, y_val),    # 学習時の"評価"につかうデータ（validation）→ early stoppingなどの指標になる
    #early_stopping_rounds=10,  # 10ラウンド経過してもeval_setに対する精度が上がらなければ打ち切り
    #use_best_model=True,       # 最も精度が高かったモデルを返す
    verbose=10
)

print("Elapsed time:",time.perf_counter()-start,"[s]")
  
gbm_results = gbm.get_evals_result()
best_iteration = gbm.get_best_iteration()  # eval_setに対するlossが最も小さかったイテレーション数

# 学習曲線の描画
loss_train = gbm_results['learn']['MultiClass']       # 訓練誤差
loss_val = gbm_results['validation']['MultiClass']    # 汎化誤差

fig = plt.figure()
ax1 = fig.add_subplot()
ax1.set_xlabel('iteration')
ax1.set_ylabel('log loss')

ax1.plot(loss_train, label='train loss')
ax1.plot(loss_val, label='val loss')
plt.legend()
plt.show()
plt.close()

print("Best iteration:", best_iteration, "train loss:", loss_train[best_iteration], "val loss:", loss_val[best_iteration])

## Step 4: 学習済みモデルを使った推論

学習済みのモデルを使って、

*   testデータセットに対する推論をして精度評価をする
*   任意の画像を入力として推論する

**validation datasetは既にハイパーパラメータの設定（best iterationの選択）に使ったので評価に使ってはいけない**



### testデータセットを使った精度評価

In [ ]:
# testデータセットからBoVWヒストグラム特徴を抽出
if os.path.exists('bovw_test.pkl'):
  print("Loading  from 'bovw_test.pkl'.")
  with open('bovw_test.pkl','rb') as f:  
    probs_test, labels_test = pickle.load(f)

else:
  print("Extracting BoVW histograms from test dataset")
  probs_test, labels_test = calc_probs(test_dataset) # TEST dataset  

  with open('bovw_test.pkl', 'wb') as f:
    pickle.dump([probs_test, labels_test], f)

# testデータセット全画像に対し推論する
print("Predicting")
start = time.perf_counter()

x_test = np.vstack(probs_test)
y_test = labels_test
y_pred = gbm.predict(x_test,ntree_start=0,ntree_end=best_iteration)  # best iterationまでの木を使う

print("Elapsed time:",time.perf_counter()-start,"[s]")

# 参考として、testとvalデータセット全画像に対しても推論する
print("Predicting for train and val datasets (for reference)")
start = time.perf_counter()

x_train = np.vstack(probs_train)
y_train = labels_train
y_pred_train = gbm.predict(x_train,ntree_start=0,ntree_end=best_iteration)  # best iterationまでの木を使う

x_val = np.vstack(probs_val)
y_val = labels_val
y_pred_val = gbm.predict(x_val,ntree_start=0,ntree_end=best_iteration)  # best iterationまでの木を使う

print("Elapsed time:",time.perf_counter()-start,"[s]")

# 精度（101クラス分類の正解率）を計算する
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

print("---Accuracy evaluation---")
print("Test accuracy:",accuracy_score(y_test,y_pred))
print("Train accuracy (for reference)':",accuracy_score(y_train,y_pred_train))
print("Val accuracy (for reference):",accuracy_score(y_val,y_pred_val))

### 任意の画像に対する予測

In [ ]:
path = "wrightflyer.jpg"

img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)

# グリッド点上に特徴点を定義
keypoints = create_dense_keypoints(img,feat,step=feat_step,scale=feat_step)
# SIFT特徴量を計算し、一番近いクラスタを見つけ（FLANNによる近似最近傍探索）、画像中の全点でヒストグラムを作る
histogram = bowExtractor.compute(img, keypoints)[0]

[[y_pred]] = gbm.predict([histogram])

print("Predicted as", categories[y_pred], ", category id:",y_pred)
imshow(img)